In [45]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 14: Docking Data ---</h2><p>As your ferry approaches the sea port, the captain asks for your help again. The computer system that runs this port isn't compatible with the docking program on the ferry, so the docking parameters aren't being correctly initialized in the docking program's memory.</p>
<p>After a brief inspection, you discover that the sea port's computer system uses a strange <a href="https://en.wikipedia.org/wiki/Mask_(computing)" target="_blank">bitmask</a> system in its initialization program. Although you don't have the correct decoder chip handy, you can emulate it in software!</p>
<p>The initialization program (your puzzle input) can either update the bitmask or write a value to memory.  Values and memory addresses are both 36-bit unsigned integers.  For example, ignoring bitmasks for a moment, a line like <code>mem[8] = 11</code> would write the value <code>11</code> to memory address <code>8</code>.</p>
<p>The bitmask is always given as a string of 36 bits, written with the most significant bit (representing <code>2^35</code>) on the left and the least significant bit (<code>2^0</code>, that is, the <code>1</code>s bit) on the right. The current bitmask is applied to values immediately before they are written to memory: a <code>0</code> or <code>1</code> overwrites the corresponding bit in the value, while an <code>X</code> leaves the bit in the value unchanged.</p>
<p>For example, consider the following program:</p>
<pre><code>mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
</code></pre>
<p>This program starts by specifying a bitmask (<code>mask = ....</code>). The mask it specifies will overwrite two bits in every written value: the <code>2</code>s bit is overwritten with <code>0</code>, and the <code>64</code>s bit is overwritten with <code>1</code>.</p>
<p>The program then attempts to write the value <code>11</code> to memory address <code>8</code>. By expanding everything out to individual bits, the mask is applied as follows:</p>
<pre><code>value:  000000000000000000000000000000001011  (decimal 11)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 00000000000000000000000000000<em>1</em>0010<em>0</em>1  (decimal 73)
</code></pre>
<p>So, because of the mask, the value <code>73</code> is written to memory address <code>8</code> instead. Then, the program tries to write <code>101</code> to address <code>7</code>:</p>
<pre><code>value:  000000000000000000000000000001100101  (decimal 101)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 00000000000000000000000000000<em>1</em>1001<em>0</em>1  (decimal 101)
</code></pre>
<p>This time, the mask has no effect, as the bits it overwrote were already the values the mask tried to set. Finally, the program tries to write <code>0</code> to address <code>8</code>:</p>
<pre><code>value:  000000000000000000000000000000000000  (decimal 0)
mask:   XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
result: 00000000000000000000000000000<em>1</em>0000<em>0</em>0  (decimal 64)
</code></pre>
<p><code>64</code> is written to address <code>8</code> instead, overwriting the value that was there previously.</p>
<p>To initialize your ferry's docking program, you need the sum of all values left in memory after the initialization program completes. (The entire 36-bit address space begins initialized to the value <code>0</code> at every address.) In the above example, only two values in memory are not zero - <code>101</code> (at address <code>7</code>) and <code>64</code> (at address <code>8</code>) - producing a sum of <em><code>165</code></em>.</p>
<p>Execute the initialization program. <em>What is the sum of all values left in memory after it completes?</em> (Do not truncate the sum to 36 bits.)</p>
</article>


In [46]:
example = """
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
"""


def initialization(s: str) -> int:
    mem = {}
    program = [l.split(" = ") for l in s.strip().splitlines()]
    mask = None
    for instruction, value in program:
        if instruction == "mask":
            mask = value
        else:
            adress = int(re.sub(r"\D", "", instruction))
            bits = f"{int(value):036b}"
            result = [None] * 36
            for i, (b, m) in enumerate(zip(bits, mask)):
                result[i] = b if m == "X" else m

            result = "".join(result)
            mem[adress] = int(result, 2)

    return sum(mem.values())


assert initialization(example) == 165

In [47]:
with open("../input/day14.txt") as f:
    puzzle = f.read()

print(f"Part I: {initialization(puzzle)}")

Part I: 13105044880745


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>13105044880745</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>For some reason, the sea port's computer system still can't communicate with your ferry's docking program. It must be using <em>version 2</em> of the decoder chip!</p>
<p>A version 2 decoder chip doesn't modify the values being written at all.  Instead, it acts as a <a href="https://www.youtube.com/watch?v=PvfhANgLrm4" target="_blank">memory address decoder</a>. Immediately before a value is written to memory, each bit in the bitmask modifies the corresponding bit of the destination <em>memory address</em> in the following way:</p>
<ul>
<li>If the bitmask bit is <code>0</code>, the corresponding memory address bit is <em>unchanged</em>.</li>
<li>If the bitmask bit is <code>1</code>, the corresponding memory address bit is <em>overwritten with <code>1</code></em>.</li>
<li>If the bitmask bit is <code>X</code>, the corresponding memory address bit is <span title="Technically, since you're on a boat, they're all floating."><em>floating</em></span>.</li>
</ul>
<p>A <em>floating</em> bit is not connected to anything and instead fluctuates unpredictably. In practice, this means the floating bits will take on <em>all possible values</em>, potentially causing many memory addresses to be written all at once!</p>
<p>For example, consider the following program:</p>
<pre><code>mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
</code></pre>
<p>When this program goes to write to memory address <code>42</code>, it first applies the bitmask:</p>
<pre><code>address: 000000000000000000000000000000101010  (decimal 42)
mask:    000000000000000000000000000000X1001X
result:  000000000000000000000000000000<em>X1</em>10<em>1X</em>
</code></pre>
<p>After applying the mask, four bits are overwritten, three of which are different, and two of which are <em>floating</em>. Floating bits take on every possible combination of values; with two floating bits, four actual memory addresses are written:</p>
<pre><code>000000000000000000000000000000<em>0</em>1101<em>0</em>  (decimal 26)
000000000000000000000000000000<em>0</em>1101<em>1</em>  (decimal 27)
000000000000000000000000000000<em>1</em>1101<em>0</em>  (decimal 58)
000000000000000000000000000000<em>1</em>1101<em>1</em>  (decimal 59)
</code></pre>
<p>Next, the program is about to write to memory address <code>26</code> with a different bitmask:</p>
<pre><code>address: 000000000000000000000000000000011010  (decimal 26)
mask:    00000000000000000000000000000000X0XX
result:  00000000000000000000000000000001<em>X</em>0<em>XX</em>
</code></pre>
<p>This results in an address with three floating bits, causing writes to <em>eight</em> memory addresses:</p>
<pre><code>00000000000000000000000000000001<em>0</em>0<em>00</em>  (decimal 16)
00000000000000000000000000000001<em>0</em>0<em>01</em>  (decimal 17)
00000000000000000000000000000001<em>0</em>0<em>10</em>  (decimal 18)
00000000000000000000000000000001<em>0</em>0<em>11</em>  (decimal 19)
00000000000000000000000000000001<em>1</em>0<em>00</em>  (decimal 24)
00000000000000000000000000000001<em>1</em>0<em>01</em>  (decimal 25)
00000000000000000000000000000001<em>1</em>0<em>10</em>  (decimal 26)
00000000000000000000000000000001<em>1</em>0<em>11</em>  (decimal 27)
</code></pre>
<p>The entire 36-bit address space still begins initialized to the value 0 at every address, and you still need the sum of all values left in memory at the end of the program.  In this example, the sum is <em><code>208</code></em>.</p>
<p>Execute the initialization program using an emulator for a version 2 decoder chip. <em>What is the sum of all values left in memory after it completes?</em></p>
</article>

</main>


In [48]:
example = """
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
"""


def memory_address_decoder(adress: int, mask: str) -> list[int]:
    def dfs(i, acc):
        if i == len(bits):
            adresses.append(int(acc, 2))
        elif mask[i] == "0":
            # If the bitmask bit is 0, the corresponding memory address bit is unchanged.
            dfs(i + 1, acc + bits[i])
        elif mask[i] == "1":
            # If the bitmask bit is 1, the corresponding memory address bit is overwritten with 1.
            dfs(i + 1, acc + "1")
        else:
            # If the bitmask bit is X, the corresponding memory address bit is floating.
            dfs(i + 1, acc + "0")
            dfs(i + 1, acc + "1")

    bits = f"{int(adress):036b}"
    adresses = []

    dfs(0, "")

    return adresses


assert memory_address_decoder(42, "000000000000000000000000000000X1001X") == [
    26,
    27,
    58,
    59,
]
assert memory_address_decoder(26, "00000000000000000000000000000000X0XX") == [
    16,
    17,
    18,
    19,
    24,
    25,
    26,
    27,
]


def initialization_v2(s: str) -> int:
    mem = {}
    program = [l.split(" = ") for l in s.strip().splitlines()]
    mask = None
    for instruction, value in program:
        if instruction == "mask":
            mask = value
        else:
            start_adress = int(re.sub(r"\D", "", instruction))
            value = int(value)

            adresses = memory_address_decoder(start_adress, mask)
            for adress in adresses:
                mem[adress] = value

    return sum(mem.values())


assert initialization_v2(example) == 208

In [49]:
print(f"Part II: {initialization_v2(puzzle)}")

Part II: 3505392154485


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3505392154485</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
